### aim: 

### date: 

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [4]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut
from scipy.interpolate import interp1d
%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [36]:
from scipy.interpolate import interp1d

In [5]:
tdir = '/gpfs/data/greenocean/software/resources/MEDUSA/PROC2/'
bgc = xr.open_dataset(f'{tdir}medusa_bc370_1y_1950_ptrc-T-CHLTADIC.nc')
ts = xr.open_dataset(f'{tdir}nemo_bc370_1y_1950_grid-T-TS.nc')


In [11]:
yr = 1950
times = pd.date_range(f"{yr}/01/01",f"{yr+1}/01/01",freq='MS',closed='left')
print(times)



savenam = 'test2.nc'
data_vars = {'vomecrty':(['time_counter', 'mean', 'stddev', 'max', 'min'], vomecrty.values,
{'units': 'µatm',
'long_name':'Surface ocean pCO2'}),
}
# define coordinates
coords = {'time_counter': (['time_counter'], time_counter),
        'time_centered': (['time_centered'], time_centered),
'nav_lat': (['y','x'], nav_lat),
'nav_lon': (['y','x'], nav_lon),
'depthv': (['depthv'], depthv)}
# define global attributes
attrs = {'made in':'SOZONE/MEDUSA/makeYearlyMEDUSAsubsetfiles.ipynb',
'desc': 'yearly medusa files, saving only variables of interest'
}
ds = xr.Dataset(data_vars=data_vars,
coords=coords,
attrs=attrs)
ds.to_netcdf(savenam)

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01', '1950-11-01', '1950-12-01'],
              dtype='datetime64[ns]', freq='MS')


In [31]:
bgc.nav_lat[0:114,250]

<xarray.DataArray 'nav_lat' (y: 114)>
array([-85.62288 , -85.47398 , -85.32379 , -85.17234 , -85.01968 , -84.86581 ,
       -84.71075 , -84.55452 , -84.39712 , -84.23854 , -84.07878 , -83.91784 ,
       -83.755684, -83.5923  , -83.42766 , -83.26174 , -83.09451 , -82.92594 ,
       -82.75598 , -82.5846  , -82.41175 , -82.23739 , -82.06146 , -81.88393 ,
       -81.70473 , -81.52381 , -81.341125, -81.15662 , -80.97021 , -80.78186 ,
       -80.5915  , -80.39907 , -80.2045  , -80.00774 , -79.8087  , -79.60733 ,
       -79.40356 , -79.19732 , -78.98854 , -78.77715 , -78.56308 , -78.34626 ,
       -78.12661 , -77.90407 , -77.67855 , -77.45    , -77.218315, -76.983444,
       -76.7453  , -76.503815, -76.2589  , -76.01049 , -75.7585  , -75.50285 ,
       -75.24347 , -74.98027 , -74.71318 , -74.44211 , -74.16698 , -73.8877  ,
       -73.6042  , -73.3164  , -73.02419 , -72.72751 , -72.426254, -72.120346,
       -71.809685, -71.494194, -71.17377 , -70.84832 , -70.51776 , -70.181984,
       -69.8409  , -69.4944  , -69.1424  , -68.7848  , -68.421486, -68.05236 ,
       -67.67733 , -67.29627 , -66.90862 , -66.51326 , -66.11152 , -65.703316,
       -65.28857 , -64.867195, -64.4391  , -64.00423 , -63.56247 , -63.113754,
       -62.658   , -62.195126, -61.725048, -61.247692, -60.762974, -60.27082 ,
       -59.77115 , -59.263885, -58.748955, -58.226284, -57.695797, -57.15743 ,
       -56.61111 , -56.05677 , -55.494347, -54.923775, -54.344994, -53.75795 ,
       -53.162582, -52.558838, -51.946674, -51.326035, -50.696884, -50.059177],
      dtype=float32)
Coordinates:
    nav_lat  (y) float32 ...
    nav_lon  (y) float32 ...
Dimensions without coordinates: y

In [32]:
bgc

<xarray.Dataset>
Dimensions:       (deptht: 75, time_counter: 12, x: 362, y: 332)
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 1950-01-31 ... 1950-12-31
    nav_lat       (y, x) float32 ...
    nav_lon       (y, x) float32 ...
  * deptht        (deptht) float32 0.50576 1.5558553 ... 5698.0605 5902.0576
Dimensions without coordinates: x, y
Data variables:
    ALK           (time_counter, deptht, y, x) float64 ...
    DIC           (time_counter, deptht, y, x) float64 ...
    CHD           (time_counter, deptht, y, x) float64 ...
    CHN           (time_counter, deptht, y, x) float64 ...
Attributes:
    made in:  SOZONE/MEDUSA/MEDUSA_PROCESSING/runner-ptrcT-TADIC.py
    desc:     yearly medusa files, saving only variables of interest

In [49]:
bgc.deptht[44:50]

<xarray.DataArray 'deptht' (deptht: 6)>
array([ 856.67896,  947.4479 , 1045.8542 , 1151.9912 , 1265.8615 , 1387.377  ],
      dtype=float32)
Coordinates:
  * deptht   (deptht) float32 856.67896 947.4479 ... 1265.8615 1387.377

In [65]:
verbose = False

### gradient

def get_grads(bgc):
    
    grad_dic_0_100_ar = np.zeros([12,114,362])
    grad_dic_0_200_ar = np.zeros([12,114,362])
    grad_dic_100_200_ar = np.zeros([12,114,362])
    grad_avg2_ar = np.zeros([12,114,362])
    DIC_1000_minus_0_ar = np.zeros([12,114,362])

    for m in range(12):
        print(m)
        for y in range(114):
            for x in range(362):


                dic_0 = bgc.DIC[m,0,y,x]
                

                if np.isnan(dic_0):
                    pass
    
                else:

                    depth_0 = bgc.deptht[0]
                    depths_first100 = bgc.deptht[0:25]
                    dics_first100 = bgc.DIC[m,0:25,y,x]
                    f = interp1d(depths_first100, dics_first100, kind='linear')
                    dic_100 = f(100)

                    depths_second100 = bgc.deptht[24:32]
                    dics_second100 = bgc.DIC[m,24:32,y,x]
                    f = interp1d(depths_second100, dics_second100, kind='linear')
                    dic_200 = f(200)

                    grad_dic_0_100 = ((dic_100-dic_0)/(100-depth_0)).values
                    grad_dic_0_200 = ((dic_200-dic_0)/(200-depth_0)).values
                    grad_dic_100_200 = ((dic_200-dic_100)/(200-100))
                    grad_avg2 = (grad_dic_100_200+grad_dic_0_100)/2

                    #bgc.deptht[44:50]
                    depths_1000 = bgc.deptht[44:50]
                    dics_1000 = bgc.DIC[m,44:50,y,x]
                    f = interp1d(depths_1000, dics_1000, kind='linear')
                    dic_1000 = f(1000)    
                    DIC_1000_minus_0 = (dic_1000-dic_0).values

                    if verbose:
                        plt.plot(bgc.DIC[m,0:32,y,x],-bgc.deptht[0:32])
                        plt.plot(dic_100, -100, 'ro')
                        plt.plot(dic_200, -200, 'r+')

                        print(f'grad_dic_0_100 {grad_dic_0_100}')
                        print(f'grad_dic_0_200 {grad_dic_0_200}')
                        print(f'grad_dic_100_200 {grad_dic_100_200}')
                        print(f'grad_avg2 {grad_avg2}')
                        print(f'DIC_1000_minus_0 {DIC_1000_minus_0}')

                    grad_dic_0_100_ar[m,y,x] = grad_dic_0_100
                    grad_dic_0_200_ar[m,y,x] = grad_dic_0_200
                    grad_dic_100_200_ar[m,y,x] = grad_dic_100_200
                    grad_avg2_ar[m,y,x] = grad_avg2_ar
    
    return grad_dic_0_100_ar, grad_dic_0_200_ar, grad_dic_100_200_ar, grad_avg2_ar


w = time.time()
grad_dic_0_100_ar, grad_dic_0_200_ar, grad_dic_100_200_ar, grad_avg2_ar = get_grads(bgc)
w2 = time.time()
print(w2-w)
# tdir = '/gpfs/data/greenocean/software/resources/MEDUSA/ukesm_allscen_gridT_TS/'
# w = xr.open_dataset(f'{tdir}/nemo_scen_{scens[s]}_1m_{y}_fy_grid-T.nc')
# tmld = w.isel(deptht=0).sel(y=slice(0,114)).weighted(mm_med['area'].sel(y=slice(0,114))).mean(dim = ['x','y'])
# vals[ind:ind+12] = tmld.thetao.values
# ind = ind+12
        
        
# grad_dic_0_100
# dic_0 = 

# grad_dic_0_200
# grad_dic_100_200



# dic_depth = bgc.DIC[0,0:31,200,200]

# # Example data arrays
# x = np.array([1, 2, 3, 4, 5])  # replace with your x values
# y = np.array([2, 3, 5, 7, 11]) # replace with your y values
# x_0 = 2.5  # replace with your target x value

# # Creating a linear interpolation function
# f = interp1d(x, y, kind='linear')

# # Interpolating to find y_0 at x_0
# y_0 = f(x_0)

0


ValueError: setting an array element with a sequence.

In [6]:
# savenam = 'test2.nc'
# data_vars = {'vomecrty':(['time_counter', 'depthv', 'y', 'x'], vomecrty.values,
# {'units': 'µatm',
# 'long_name':'Surface ocean pCO2'}),
# }
# # define coordinates
# coords = {'time_counter': (['time_counter'], time_counter),
#         'time_centered': (['time_centered'], time_centered),
# 'nav_lat': (['y','x'], nav_lat),
# 'nav_lon': (['y','x'], nav_lon),
# 'depthv': (['depthv'], depthv)}
# # define global attributes
# attrs = {'made in':'SOZONE/MEDUSA/makeYearlyMEDUSAsubsetfiles.ipynb',
# 'desc': 'yearly medusa files, saving only variables of interest'
# }
# ds = xr.Dataset(data_vars=data_vars,
# coords=coords,
# attrs=attrs)
# ds.to_netcdf(savenam)